In [116]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

In [11]:
client_id = "zuo60iNcVNsA5BmyfHL64nzVMAfZ-suiFONibf8_EjOPXeUQgrIqX-w3CA-RRji8"
client_secret = "KsD2On2pG2zR7vgvTLkp7zW5i3LHoKuIB8OqyN0uV-AAIocMwwi8JQTeaolFozKSvQpiQkH6V0wGh6Jr7qSW1w"
token = "_Q2Mi2ftYFYIkFAUPY9lfkROyNXdj5ML8_6riblV-YCcfHvEvhy5k5Ax5zcWvd-E"
artist_name = 'taylor swift'

###  Function to call the API for Genius, a music information website, using the requests module. First, the function calls to get Swift’s general search information. From this it extracts her artist ID, which is needed to make a call to get her song names and song IDs. Then, to classify the songs by album, it makes calls using each song’s ID. Note: the function can only return one dataframe per call.

In [146]:
def make_call(show_songs_df, lyrics, albums):
    
    url = "https://api.genius.com/search"
    payload = {'access_token': token, 'q': artist_name}
    r = requests.get(url, params=payload)
    search_response = r.json()
    artist_id = search_response['response']['hits'][0]['result']['primary_artist']['id']
    
    page_number = 1
    song_names_ids = []
    while True:
        url2 = "https://api.genius.com/artists/"
        payload2 = {'access_token': token,
                    'sort': 'popularity',
                    'per_page': 50,
                    'page': page_number}
        r2 = requests.get(url2 + str(artist_id) + "/songs", params=payload2)
        artist_songs_response = r2.json()
        songs = artist_songs_response['response']['songs']

        for song in songs:
            new_song = {}
            new_song['title'] = song['title']
            new_song['song_id'] = song['id']
            song_names_ids.append(new_song)

        if len(songs) < 50:
            break
        page_number += 1
    
    if show_songs_df == True:
        song_names_ids_df = pd.DataFrame(data=song_names_ids)
        return song_names_ids_df
    
    if lyrics == True:
        songs_and_lyrics = {}
        for song in song_names_ids:
            api_url = "https://api.genius.com/songs/"
            payload = {'access_token': token}
            r = requests.get(api_url + str(song['song_id']), params=payload)
            song_response = r.json()
            lyrics_path = song_response['response']['song']['path']
    
            page_url = "http://genius.com" + lyrics_path
            page = requests.get(page_url)
            html = BeautifulSoup(page.text, 'html.parser')
#             [h.extract() for h in html('script')]
            
            songs_and_lyrics[song['title']] = []
            lyrics_html = html.find('div', class_='lyrics')
            if lyrics_html is not None:
                retrieved_lyrics = lyrics_html.get_text()
                songs_and_lyrics[song['title']].append(retrieved_lyrics)
#                 retrieved_lyrics = lyrics_html.get_text()
#             except AttributeError:
#                 print("AttributeError")
            
#             songs_and_lyrics[song['title']] = retrieved_lyrics
    
#         songs_and_lyrics_df = pd.DataFrame.from_records(data=songs_and_lyrics, index=list(range(0,249))
        print(songs_and_lyrics)
    
    if albums == True:
        songs_and_albums = []
        for song in song_names_ids:
            new_song = {}
            url3 = "https://api.genius.com/songs/"
            payload3 = {'access_token': token}
            r3 = requests.get(url3 + str(song['song_id']), params=payload3)
            song_album_response = r3.json()
            new_song['song'] = song_album_response['response']['song']['title']
            try:
                new_song['album'] = song_album_response['response']['song']['album']['name']
            except:
                new_song['album'] = "Unknown"
            songs_and_albums.append(new_song)
            
        filename = 'songs_and_albums.json'
        with open(filename, 'w') as f_obj:
            json.dump(songs_and_albums, f_obj)

        songs_and_albums_df = pd.DataFrame(data=songs_and_albums)
        return songs_and_albums_df

### First, I execute the make_call() function to return the dataframe of song names and song IDs.

In [42]:
song_names_ids_df = make_call(show_songs_df=True, albums=False)

In [43]:
song_names_ids_df

,title,song_id
0,​cardigan,5793984
1,​exile,5793983
2,Lover,4508914
3,​the 1,5794073
4,Look What You Made Me Do,3210592
...,...,...
244,I’d Lie,187250
245,Forever & Always (Piano Version) [Taylor’s Ver...,6478860
246,Brought Up That Way,187116
247,I Heart ?,187203


### I save this dataframe to a CSV file in order to store it in my PostgreSQL database.

In [20]:
song_names_ids_df.to_csv('song_names_ids.csv',index=False)

### Then I execute the make_call() function to return the dataframe of songs and albums.

In [38]:
songs_and_albums_df = make_call(show_songs_df=False, albums=True)

In [44]:
songs_and_albums_df

,song,album
0,​cardigan,folklore
1,​exile,folklore
2,Lover,Lover
3,​the 1,folklore
4,Look What You Made Me Do,reputation
...,...,...
244,I’d Lie,Unreleased Songs
245,Forever & Always (Piano Version) [Taylor’s Ver...,Fearless (Taylor’s Version)
246,Brought Up That Way,Unreleased Songs
247,I Heart ?,Taylor Swift (Best Buy Exclusive)


### I also save this dataframe to a CSV file to store it in my PostgreSQL database.

In [30]:
songs_and_albums_df.to_csv('songs_and_albums.csv',index=False)

### I iterate through this dataframe to create one long list of all the song names. 

In [45]:
song_names = []
for index in range(len(songs_and_albums_df)):
    song = songs_and_albums_df['song'].iloc[index]
    song_names.append(song)
print(song_names)

['\u200bcardigan', '\u200bexile', 'Lover', '\u200bthe 1', 'Look What You Made Me Do', '\u200bbetty', 'End Game', '\u200baugust', 'You Need To Calm Down', 'ME!', 'Blank Space', '...Ready for It?', '\u200bchampagne problems', '\u200bmy tears ricochet', '\u200bwillow', '\u200binvisible string', '\u200bthe last great american dynasty', 'Cruel Summer', 'Delicate', 'Call It What You Want', 'Bad Blood (Remix)', 'Style', '\u200bseven', 'I Don’t Wanna Live Forever', '\u200billicit affairs', '\u200bthis is me trying', 'Love Story', 'Gorgeous', '\u200bevermore', '\u200bhappiness', '\u200bpeace', '\u200bmad woman', '\u200bmirrorball', 'The Man', 'Don’t Blame Me', 'I Did Something Bad', 'Dress', '\u200btolerate it', '\u200bno body, no crime', '\u200bhoax', '\u200bgold rush', 'Getaway Car', '\u200bivy', '\u200bepiphany', 'The Archer', 'All Too Well', 'Mr. Perfectly Fine (Taylor’s Version) [From the Vault]', 'Bad Blood', 'Miss Americana & The Heartbreak Prince', 'London Boy', 'Cornelia Street', 'Wild

### The list of song names is in Unicode, so I convert it to ASCII and decode it.

In [46]:
converted_song_names = []
for song_name in song_names:
    converted_song_name = song_name.encode('ascii', 'ignore').decode()
    converted_song_names.append(converted_song_name)
print(converted_song_names)

['cardigan', 'exile', 'Lover', 'the 1', 'Look What You Made Me Do', 'betty', 'End Game', 'august', 'You Need To Calm Down', 'ME!', 'Blank Space', '...Ready for It?', 'champagne problems', 'my tears ricochet', 'willow', 'invisible string', 'the last great american dynasty', 'Cruel Summer', 'Delicate', 'Call It What You Want', 'Bad Blood (Remix)', 'Style', 'seven', 'I Dont Wanna Live Forever', 'illicit affairs', 'this is me trying', 'Love Story', 'Gorgeous', 'evermore', 'happiness', 'peace', 'mad woman', 'mirrorball', 'The Man', 'Dont Blame Me', 'I Did Something Bad', 'Dress', 'tolerate it', 'no body, no crime', 'hoax', 'gold rush', 'Getaway Car', 'ivy', 'epiphany', 'The Archer', 'All Too Well', 'Mr. Perfectly Fine (Taylors Version) [From the Vault]', 'Bad Blood', 'Miss Americana & The Heartbreak Prince', 'London Boy', 'Cornelia Street', 'Wildest Dreams', 'tis the damn season', '1 step forward, 3 steps back', 'King of My Heart', 'Daylight', 'Death by a Thousand Cuts', 'the lakes', 'I For

In [147]:
songs_and_lyrics = make_call(show_songs_df=False, lyrics=True, albums=False)

UnboundLocalError: local variable 'retrieved_lyrics' referenced before assignment

In [103]:
with open('lyrics_1.json', 'w') as f:
        json.dump(songs_and_lyrics, f)

In [ ]:
songs_and_lyrics_df.to_csv('songs_and_lyrics.csv',index=False)

In [ ]:
https://api.genius.com/songs/378195?access_token=_Q2Mi2ftYFYIkFAUPY9lfkROyNXdj5ML8_6riblV-YCcfHvEvhy5k5Ax5zcWvd-E

In [ ]:
songs_and_lyrics = {}
for song in song_names_ids:
    api_url = "https://api.genius.com/songs/"
    payload = {'access_token': token}
    r = requests.get(api_url + str(song['song_id']), params=payload)
    if r.status_code == 200:
        song_response = r.json()
        lyrics_path = song_response['response']['song']['path']
    
        page_url = "http://genius.com" + lyrics_path
        page = requests.get(page_url)
        if page.status_code == 200:
            html = BeautifulSoup(page.text, "html.parser")
            [h.extract() for h in html('script')]
            lyrics = html.find(“div”, class_=”lyrics”).get_text()
            songs_and_lyrics[song.lower()] = lyrics
    
songs_and_lyrics_df = pd.DataFrame(data=songs_and_lyrics)
return songs_and_lyrics_df
            
        
        
        

In [51]:
def get_lyrics(converted_song_names):
    songs_and_lyrics = {}
    for song_name in converted_song_names:
        url = 'https://api.lyrics.ovh/v1/Taylor Swift/' + song_name
        lyrics_response = requests.get(url)
        if lyrics_response.status_code == 200:
            lyrics = json.loads(lyrics_response.text)['lyrics']
            ascii_lyrics = lyrics.encode('ascii', 'replace').decode()
            filtered_lyrics = ascii_lyrics.replace("\n", " ").replace("\r", " ").replace(',', ' ').replace('?', ' ').replace('(', ' ').replace(')', ' ').replace('.', ' ').lower()
            lyric_words = [word for word in filtered_lyrics.split(" ") if word != ""]
            songs_and_lyrics[song_name.lower()] = lyric_words
    
    songs_and_lyrics_df = pd.DataFrame(data=songs_and_lyrics)
    return songs_and_lyrics_df

In [54]:
songs_and_lyrics_df.to_csv('songs_and_lyrics.csv',index=False)

In [40]:
filename = 'songs_and_albums.json'
with open(filename) as f_obj:
    songs_and_albums = json.load(f_obj)
new_songs_and_albums = []
for song_album in songs_and_albums:
    new_song_album = {"song": song_album["song"].encode('ascii', 'ignore').decode(),
                        "album": song_album["album"].encode('ascii', 'ignore').decode()}
    new_songs_and_albums.append(new_song_album)